In [18]:
import json
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
import cv2
from utils import *

In [20]:
for i in range(5):    
    video_path = f'/media/kyrollosd/FAA/Cabin/rowL-{i}.mp4'

    #seat detect
    with open(f"{video_path}.json",'r') as f:
        ann = json.load(f)
        bbox_seat = []
        for a in ann["shapes"]:
            if a['label'] == 'seat':
                im = np.zeros((ann['imageHeight'],ann['imageWidth']),dtype=bool)
                bbox = np.array(a['points']).astype(int).flatten()
                bbox_seat.append(bbox)
                        
        bbox_seat = np.stack(bbox_seat,axis=0)

    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    nb_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = int(fps)*10
    curr = int(video.get(cv2.CAP_PROP_POS_FRAMES))

    while True:
        ret, frame = video.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        for j in range(3):
            bbox = bbox_seat[j]
            bbox_shape = [bbox[2]+1-bbox[0],bbox[3]+1-bbox[1]]
            max_dim = np.argmax(bbox_shape)
            min_dim = np.argmin(bbox_shape)
            exp = (bbox_shape[max_dim]-bbox_shape[min_dim])//2
            if max_dim == 1:
                newbbox = [exp,0,bbox_shape[min_dim]+exp,bbox_shape[max_dim]]
            else:
                newbbox = [0,exp,bbox_shape[max_dim],bbox_shape[min_dim]+exp]
            square = np.zeros((bbox_shape[max_dim],bbox_shape[max_dim],3),dtype=int)
            square[newbbox[1]:newbbox[3],newbbox[0]:newbbox[2]] = frame[bbox[1]:bbox[3]+1,bbox[0]:bbox[2]+1]
            cv2.imwrite(f'seat/{i}_{j}_{curr}.jpg',square[...,::-1])
        curr += frame_interval
        video.set(cv2.CAP_PROP_POS_FRAMES,curr)
    
    video.release()

In [12]:
40*60*3*12

86400